In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import MeCab
import ipadic
import polars as pl

In [3]:
import sys

# sys.path.append('/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky')
sys.path.append('/home/jovyan/nas/medinfo_lab/Research_Projects/zhang/Husky')

In [4]:
# import tools.tokenizers.myMeCab

In [6]:
# from tools.tokenizers import myMeCab

In [7]:
TokenizerPath = '/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky/kuhpTokenizer/40000_vocab/bert_bpe_vocab_40000.model'.replace("mi-drive", "nas")
# CorpusPath = '/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky/data/del_none_data_for_train_tokenizer.txt'.replace("mi-drive", "nas")
CorpusPath = '/home/jovyan/data/del_none_data_for_train_tokenizer.txt'

In [8]:
!ls -lha $CorpusPath

-rwxr-xr-x 1 jovyan users 9.6G Dec  7 15:25 /home/jovyan/data/del_none_data_for_train_tokenizer.txt


In [9]:
# 10001 % 1000

In [10]:
DICT_PATH

'/home/jovyan/nas/medinfo_lab/Research_Projects/zhang/Husky/tools/data/MANBYO_202106.dic'

In [13]:
import re
import sys

# sys.path.append('/home/jovyan/mi-drive/medinfo_lab/Research_Projects/zhang/Husky')
sys.path.append('/home/jovyan/nas/medinfo_lab/Research_Projects/zhang/Husky')
# dependence
import MeCab
import json
import ipadic
import six
from tools import DICT_PATH, STOP_WORDS

# PATH
SPACES_PATTERN = r"[\s\n\r]+"
SPACES_RE = re.compile(SPACES_PATTERN)

with open(STOP_WORDS, 'r', encoding='UTF-8') as file:
     stopwords = json.load(file)
tagger = MeCab.Tagger(ipadic.MECAB_ARGS + f" -O wakati {DICT_PATH}")

def tokenizex(text, stemmer):
    text = text.lower()
    vail_tokens = []
    _tokens = SPACES_RE.split(text)
    if stemmer:
        _tokens = [six.ensure_str(stemmer.stem(x)) if len(x) > 3 else x for x in _tokens]
    # with open(STOP_WORDS, 'r', encoding='UTF-8') as file:
    #     stopwords = json.load(file)

    # tagger = MeCab.Tagger(ipadic.MECAB_ARGS + f" -O wakati -u {DICT_PATH}")
    for line in _tokens:
        tokens = tagger.parse(line)
        if tokens is None:
            continue
        else:
            tokens = tokens.split()
            tokens = [token for token in tokens if token not in stopwords]
            vail_tokens.extend(tokens)

    return vail_tokens


def tokenizey(text, stemmer):
    text = text.lower()
    vail_tokens = []
    _tokens = SPACES_RE.split(text)
    if stemmer:
        _tokens = [six.ensure_str(stemmer.stem(x)) if len(x) > 3 else x for x in _tokens]
    # with open(STOP_WORDS, 'r', encoding='UTF-8') as file:
    #     stopwords = json.load(file)

    # tagger = MeCab.Tagger(ipadic.MECAB_ARGS + f" -O wakati -u {DICT_PATH}")
    for line in _tokens:
        tokens = tagger.parse(line)
        if tokens is None:
            continue
        else:
            tokens = tokens.split()
            tokens = [token for token in tokens if token not in stopwords]
            tokens = list(set(tokens))
            vail_tokens.extend(tokens)

    return vail_tokens


In [12]:
# stopwords

In [24]:
from collections import Counter

def lines2tokendict(lines):
    l_tokens = []
    for s in lines:
        l_tokens.extend(tokenizex(text=s, stemmer=False))
    return Counter(l_tokens)

def lines2docdict(lines):
    l_tokens = []
    for s in lines:
        l_tokens.extend(tokenizey(text=s, stemmer=False))
    return Counter(l_tokens)

In [17]:
# %%time
# tfidf_vectorizer = TfidfVectorizer(use_idf=True, tokenizer=tokenizex, input="filename")

# # tfidf_matrix = tfidf_vectorizer.fit_transform(data)
# tfidf_matrix = tfidf_vectorizer.fit_transform([CorpusPath])

In [18]:
%%time
# with open(CorpusPath, 'r', encoding='utf-8') as f:
# #     for i, s_line in enumerate(f):
# #         if (i+1) % 10000000 == 0:
# #             print(i)
#     # res_list = Parallel(n_jobs=-1, verbose=5, backend="multiprocessing")(delayed(line2token)(l) for l in f)

CPU times: user 2 µs, sys: 4 µs, total: 6 µs
Wall time: 11.7 µs


In [23]:
# def load_data(CorpusPath):
#     l_d = []
#     with open(CorpusPath, 'r', encoding='utf-8') as f:
#         l_text = []
#         for i, l in enumerate(f):
#             l_text.append(l)
#             # print(l)
#             # break
#             if (i+1) % 1000 == 0:
#                 l_d.append(lines2tokendict(l_text))

#                 l_text = []
                
#             if (i+1) % 100000 == 0:
#                 print(i)

#             # if (i+1) == 3000:
#             #     break
#     return l_d

In [21]:
import subprocess

def get_line_count(filename):
    output = subprocess.check_output(["wc", "-l", filename])
    return int(output.decode("utf-8").split()[0])

len_lines_whole = get_line_count(CorpusPath)
print(len_lines_whole)

169827322


In [25]:
%%time
import os
from multiprocessing import Pool
from pathlib import Path

IN = Path(CorpusPath)

def run(start: int, end: int):
    current = start
    
    l_text = []
    l_t = []
    l_d = []
    
    with IN.open() as f:
        f.seek(start)

                        
        for i, line in enumerate(f):
            current += len(line.encode())
            line = line.strip()
                
            l_text.append(line)
            # print(l)
            # break
            if (i+1) % 1000 == 0:
                l_t.append(lines2tokendict(l_text))
                l_d.append(lines2docdict(l_text))

                l_text = []
                #print(i)
                
                
                
            if current >= end:
                break
        t_whole = reduce(add, l_t)
        d_whole = reduce(add, l_d)
    return t_whole, d_whole



file_size = IN.stat().st_size
# num_procs = os.cpu_count()
num_procs = 512

chunk_size = file_size // num_procs
chunks = []
start, end = 0, chunk_size

with IN.open(encoding="utf-8", errors="ignore") as f:
    while end < file_size:
        f.seek(end)
        f.readline()

        end = f.tell()
        chunks.append((start, end))
        start, end = end, end + chunk_size

    chunks.append((start, file_size))

    


CPU times: user 28.7 ms, sys: 402 µs, total: 29.1 ms
Wall time: 27.3 ms


In [31]:
from functools import reduce
from operator import add


In [32]:
import joblib
from joblib import Parallel, delayed

In [33]:
%%time
res_list = Parallel(n_jobs=-1, verbose=10, backend="multiprocessing")(delayed(run)(i[0], i[1]) for i in chunks)

l_t = []
l_d = []

for r in res_list:
    l_t.append(r[0])
    l_d.append(r[1])
# with Pool(processes=num_procs) as pool:
#     for d_sub in pool.starmap(run, chunks):
#         res_list.append(d_sub)

t_whole = reduce(add, l_t)
d_whole = reduce(add, l_d)

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   57.1s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:   58.6s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 274 tasks      |

CPU times: user 4min 5s, sys: 4.9 s, total: 4min 10s
Wall time: 11min 38s


In [34]:
import pickle

def d2p(d, f):
    with open(f, 'wb') as file:
        pickle.dump(d, file)


def p2d(f):
    with open(f, 'rb') as file:
        d = pickle.load(file)
    return d

In [40]:
%%time

d2p(t_whole, "./whole_t_count.pkl")
d2p(d_whole, "./whole_d_count.pkl")


CPU times: user 393 ms, sys: 24.1 ms, total: 417 ms
Wall time: 637 ms


dict about token count in whole text

In [36]:
for i, (k,v) in enumerate(t_whole.items()):
    print(k, v)
    if i == 10:
        break

→ 8562108
確認 2200699
2 18866482
nd 83273
後 6736704
3 13155441
m 1878588
以降 557666
af 227995
再発 1241549
日 8616066


dict about document count about tokents

In [37]:
for i, (k,v) in enumerate(d_whole.items()):
    print(k, v)
    if i == 10:
        break

→ 7811666
確認 2124621
m 1829484
化 1048064
時期 165045
3 12119719
後 6411274
再発 1206246
以降 550198
日 7764575
大体 8729


In [39]:
d1 = lines2tokendict(["クラリシッドで下痢","カルボシステインで湿疹", "小青竜湯で胃の重さ"])
d1

Counter({'クラリシッド': 1,
         '下痢': 1,
         'カルボシステイン': 1,
         '湿疹': 1,
         '小': 1,
         '青': 1,
         '竜': 1,
         '湯': 1,
         '胃': 1,
         '重': 1})

In [ ]:
d1 = lines2tokendict(["クラリシッドでクラリシッドの下痢","カルボシステインで湿疹がカルボシステインに湿疹", "小青竜湯で胃の重さに胃が重い"])

In [23]:
d2 = lines2tokendict(["アンブロキソールで途中覚醒","カルボシステインで湿疹", "小青竜湯で胃の重さ"])

In [ ]:
from tools.valuations.entropy import calculate_entropy

In [ ]:
output = []
sentence_matrix = tfidf_vectorizer.transform(data[:1000]).toarray()
for index, line in enumerate(data[:1000]):
    ens = calculate_entropy(sentence_matrix, batch=True)
    output.append(ens[index], line)

In [ ]:
import matplotlib.pyplot as plt
entropy = []
for item in output:
    entropy.append(item[0])
plt.hist(entropy, bins=80)
plt.xlabel('entropy')
plt.ylabel('count')
plt.title('entropy distribute Plot')
plt.show()